In [4]:
# ----------------------------
# Import libraries
# ----------------------------
from shiny import App, ui, render, reactive
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from PIL import Image
import shutil
import os
from tensorflow.keras.preprocessing import image  # Used for image to array conversion and preprocessing

import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# Upload the image saving path
# ----------------------------
UPLOAD_SAVE_DIR = "/Users/homeyellow/Desktop/uploads"

# ----------------------------
# List of model names (for dropdown selection)
# ----------------------------
MODEL_NAMES = [
    "VGG16",
    "VGG19",
    "CNN (50x50)",
    "CNN (100x100)",
    "InceptionV3"
]

# ----------------------------
# The image is cut into patches
# ----------------------------
def split_image_to_patches(img, patch_size=(50, 50)):
    width, height = img.size
    patches, positions = [], []
    for y in range(0, height, patch_size[1]):
        for x in range(0, width, patch_size[0]):
            box = (x, y, x + patch_size[0], y + patch_size[1])
            patch = img.crop(box)
            patches.append(patch)
            positions.append((x, y))
    return patches, positions

# ----------------------------
# Preprocess all patches
# ----------------------------
def preprocess_patches(patches, preprocess_fn, target_size):
    processed = []
    for patch in patches:
        patch = patch.resize(target_size)
        arr = image.img_to_array(patch)
        arr = np.expand_dims(arr, axis=0)
        arr = preprocess_fn(arr)
        processed.append(arr[0])
    return np.array(processed)

# ----------------------------
# Restore the patch prediction to a heat map
# ----------------------------
def create_prediction_grid(positions, classes, img_size, patch_size):
    grid_w = img_size[0] // patch_size[0]
    grid_h = img_size[1] // patch_size[1]
    heatmap = np.zeros((grid_h, grid_w))
    for (x, y), cls in zip(positions, classes):
        row = y // patch_size[1]
        col = x // patch_size[0]
        heatmap[row, col] = cls
    return heatmap

# ----------------------------
# Front-end UI construction
# ----------------------------
app_ui = ui.page_fluid(
    ui.panel_title("🧠 ML Classifier Dashboard"),
    ui.navset_card_tab(

        # 📘 About page
        ui.nav_panel("📖 About",
            ui.card(
                ui.h2("📘 User Guide", class_="text-primary"),
                ui.markdown("""
                **How to use this dashboard:**
                - Step 1: Go to the 'Models' tab to select a classifier.
                - Step 2: Adjust parameters and view CV results (heatmap, metrics, distributions).
                - Step 3: Use the 'Image Result' tab to upload an image and see predictions.
                """),
                ui.hr(),
                ui.h2("📊 Data Dictionary", class_="text-success"),
                ui.markdown("""
                - Synthetic Dataset: Generated with sklearn, 2 informative features.
                - Metrics:
                    - Accuracy, F1 Score across CV folds
                    - Confusion Matrix
                    - Predicted label distributions and Chi-squared statistic
                    - Outer model summary: % Cancer Cells, Largest Tumor Mass
                """),
                full_screen=True
            )
        ),

        # 🧠 Models page（Inner Model）
        ui.nav_panel("🧠 Models",
            ui.layout_sidebar(
                ui.sidebar(
                    ui.input_select("model_select", "Select Classifier", MODEL_NAMES),
                    ui.input_slider("n_samples", "Number of Samples", 50, 500, 150, step=50),
                    ui.input_slider("noise", "Label Noise", 0.0, 0.4, 0.1, step=0.05),
                    ui.output_ui("model_params_ui"),  # Dynamic parameters（如 dropout、fine-tune 层数）
                    ui.hr(),
                    ui.h5("📉 Model Confidence", class_="text-muted"),
                    ui.output_text("error_display")
                ),
                ui.layout_columns(
                    ui.card(ui.h4("📌 Heatmap from CV"), ui.output_plot("decision_boundary_plot"), full_screen=True),
                    ui.card(ui.h4("📈 Performance Metrics"), ui.output_plot("cv_metrics_plot"), full_screen=True),
                    ui.card(ui.h4("📊 Distributions + Chi-Squared"), ui.output_plot("cv_distribution_plot"), full_screen=True),
                    ui.card(ui.h4("📋 Outer Model Metrics"), ui.output_text("outer_model_metrics"), full_screen=True),
                    col_widths=[6, 6]
                )
            )
        ),

        # 🖼️ Image Result page（Outer Model）
        ui.nav_panel("🖼️ Image Test",
            ui.layout_sidebar(
                ui.sidebar(
                    ui.input_select("upload_model_select", "Select Classifier", MODEL_NAMES),
                    ui.input_file("image_upload", "Upload an Image", accept=[".jpg", ".jpeg", ".png"]),
                    ui.output_text("show_uploaded_info"),
                    ui.hr(),
                    ui.h5("📉 Model Confidence", class_="text-muted"),
                    ui.output_text("upload_model_confidence")
                ),
                ui.layout_columns(
                    ui.card(ui.h4("🧯 Full Image Heatmap"), ui.output_plot("upload_heatmap"), full_screen=True),
                    ui.card(ui.h4("🧪 Inner Model Prediction Distribution"), ui.output_plot("upload_distribution"), full_screen=True),
                    ui.card(ui.h4("🧾 Outer Model Summary"), ui.output_text("upload_outer_metrics"), full_screen=True),
                    col_widths=[6, 6]
                )
            )
        )
    )
)

# ----------------------------
# Back-end logic
# ----------------------------
def server(input, output, session):

    # 🔴 TODO: Inner Model loading (reserve 5 model positions)
    @reactive.calc
    def get_model():
        name = input.model_select()
        # 🔴 TODO: Load and return the built-in models (such as CNN, VGG)
        if name == "VGG16":
            # from tensorflow.keras.applications import VGG16
            # return VGG16(...) or load_model("vgg16_inner.h5")
            pass
        elif name == "VGG19":
            pass
        elif name == "CNN (50x50)":
            pass
        elif name == "CNN (100x100)":
            pass
        elif name == "InceptionV3":
            pass

        return LogisticRegression()  # Occupying position model

    # Dynamic display model parameter slider (applicable to CNN/VGG)
    @output
    @render.ui
    def model_params_ui():
        name = input.model_select()
        if name in ["CNN (50x50)", "CNN (100x100)"]:
            return ui.input_slider("dropout_rate", "Dropout Rate", 0.0, 0.5, 0.2, step=0.05)
        elif name in ["VGG16", "VGG19"]:
            return ui.input_slider("fine_tune_layers", "Unfreeze Last N Layers", 0, 10, 0, step=1)
        return ui.p("No tunable hyperparameters for this model.")

    # The simulated dataset generated using sklearn
    @reactive.calc
    def dataset():
        X, y = make_classification(
            n_samples=input.n_samples(),
            n_features=2,
            n_informative=2,
            n_redundant=0,
            flip_y=input.noise(),
            random_state=42
        )
        return X, y

    # Display the cross-validation heat map (decision boundary)
    @output
    @render.plot
    def decision_boundary_plot():
        X, y = dataset()
        model = get_model()
        model.fit(X, y)

        h = 0.02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
        plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', cmap=plt.cm.coolwarm)
        plt.title(f"{input.model_select()} CV Heatmap")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")

    # Display the Accuracy/F1 distribution graph of cross-validation
    @output
    @render.plot
    def cv_metrics_plot():
        accs, f1s = [], []
        X, y = dataset()
        model = get_model()
        for _ in range(5):
            idx = np.random.permutation(len(X))
            split = int(0.8 * len(X))
            model.fit(X[idx[:split]], y[idx[:split]])
            pred = model.predict(X[idx[split]:])
            accs.append(accuracy_score(y[idx[split]:], pred))
            f1s.append(random.uniform(0.6, 0.95))  # 模拟 F1

        df = pd.DataFrame({'Accuracy': accs, 'F1 Score': f1s})
        sns.boxplot(data=df)
        plt.title("Cross-Validation Metrics")

    # Display the randomly generated distribution map (simulation)
    @output
    @render.plot
    def cv_distribution_plot():
        dist = np.random.poisson(5, 100)
        sns.histplot(dist, kde=True)
        plt.title("Prediction Distributions")

    @output
    @render.text
    def outer_model_metrics():
        return "\n".join([
            "Outer Model Evaluation:",
            " - Cancer Cell %: XX.XX%",
            " - Largest Tumor Mass Area: XXX sq px",
            " - Model Uncertainty: ±XX.XX%"
        ])

    @output
    @render.text
    def error_display():
        return "Estimated model error range: ±XX.XX%"

    # 🔴 TODO: Outer Model loading and prediction (The Patch cutting logic has been completed and is awaiting integration into the model.）
    @output
    @render.plot
    def upload_heatmap():
        if input.image_upload():
            filepath = input.image_upload()[0]['datapath']
            try:
                model, preprocess, target_size = upload_model()
                if model is None:
                    raise ValueError("Model not loaded.")
                img = Image.open(filepath).convert("RGB")
                patch_size = target_size
                patches, positions = split_image_to_patches(img, patch_size=patch_size)
                patch_batch = preprocess_patches(patches, preprocess, patch_size)
                preds = model.predict(patch_batch)
                classes = (
                    np.argmax(preds, axis=1)
                    if preds.shape[1] > 1
                    else (preds.flatten() > 0.5).astype(int)
                )
                heatmap = create_prediction_grid(positions, classes, img.size, patch_size)
                plt.imshow(heatmap, cmap="hot", interpolation="nearest")
                plt.title("Predicted Heatmap (by patch)")
                plt.axis("off")
            except Exception as e:
                plt.text(0.5, 0.5, f"Error: {e}", ha='center', va='center')
                plt.axis('off')
        else:
            img = np.random.rand(10, 10)
            plt.imshow(img, cmap='hot')
            plt.title("Placeholder Heatmap")
            plt.axis('off')

    @output
    @render.plot
    def upload_distribution():
        values = np.random.normal(50, 15, 100)
        sns.histplot(values, bins=10, kde=True)
        plt.title("Inner Model Prediction Distribution")

    @output
    @render.text
    def upload_outer_metrics():
        return "\n".join([
            "Uploaded Image Outer Model:",
            " - Cancer Cell %: XX.XX%",
            " - Largest Tumor Mass: XXX px",
            " - Confidence: High"
        ])

    @output
    @render.text
    def upload_model_confidence():
        return "Estimated error: ±XX.XX%"

    # The information will be displayed after uploading the picture
    @output
    @render.text
    def show_uploaded_info():
        if input.image_upload():
            fileinfo = input.image_upload()[0]
            saved_name = fileinfo['name']
            temp_path = fileinfo['datapath']
            size_mb = fileinfo['size'] / (1024 ** 2)
            if not os.path.exists(UPLOAD_SAVE_DIR):
                os.makedirs(UPLOAD_SAVE_DIR)
            final_save_path = os.path.join(UPLOAD_SAVE_DIR, saved_name)
            try:
                shutil.copy(temp_path, final_save_path)
                return f"""✅ Uploaded File Info:
- File Name: {saved_name}
- Temp Path: {temp_path}
- Size: {size_mb:.2f} MB
- ✅ Saved To: {final_save_path}
                """
            except Exception as e:
                return f"❌ Error saving file: {e}"
        return "📂 Please upload an image to see info and save it."

    # 🔴 TODO: Outer Model loading (reserve 5 models.）
    @reactive.calc
    def upload_model():
        name = input.upload_model_select()
        # 🔴 TODO: Replace this with actual model loading logic
        # if name == "VGG16": return load_model("vgg16.h5"), preprocess_input, (224, 224)
        return None, lambda x: x / 255.0, (50, 50)  # CNN(50x50) Occupy position



app = App(app_ui, server)


ModuleNotFoundError: No module named 'shiny'